In [1]:
def drawPolyline(im, landmarks, start, end, isClosed=False) :
    points = []
    for i in range(start, end+1) :
        point = [landmarks.part(i).x, landmarks.part(i).y]
        points.append(point)
        
    points = np.array(points, dtype=np.int32)
    cv2.polylines(im, [points], isClosed, (255, 200, 0),
                 thickness=2, lineType=cv2.LINE_8)
    

# Use this function for 70-poinst facial lanmark detector model
def renderFace(im, landmarks) :
    assert(landmarks.num_parts == 68)
    drawPolyline(im, landmarks, 0, 16)              # Jaw line
    drawPolyline(im, landmarks, 17, 21)             # Left eyebrow
    drawPolyline(im, landmarks, 22, 26)             # Right eyebrow
    drawPolyline(im, landmarks, 27, 30)             # NOse bridge 
    drawPolyline(im, landmarks, 30, 35, True)       # Lower nose
    drawPolyline(im, landmarks, 36, 41, True)       # Left eye
    drawPolyline(im, landmarks, 42, 47, True)       # Right eye
    drawPolyline(im, landmarks, 48, 59, True)       # Outer LIp
    drawPolyline(im, landmarks, 60, 67, True)       # Inner lip

    
# Ues this funciton for any model other than 
# 70 points facial_landmark detctor model
def renderFace2(im, landmarks, color=(0, 255, 0), radius=3):
    for p in landmarks.parts() :
        cv2.circle(im, (p.x, p.y), radius, color, -1)

In [2]:
import dlib, cv2
import numpy as np
# from renderFace import renderFace

def writeLandmarksToFile(landmarks, landmarksFileName) :
    with open(landmarksFileName, 'w') as f:
        for p in landmarks.parts() :
            f.write("%s %s\n" %(int(p.x), int(p.y)))
            
    f.close()

In [3]:
# Landmark model location
PREDICTOR_PATH = "./models/shape_predictor_68_face_landmarks.dat"

# Get the face detector
faceDetector = dlib.get_frontal_face_detector()

# The landmark detector is implemented in the shape_predictor class
landmarkDetector = dlib.shape_predictor(PREDICTOR_PATH)


In [13]:
# Read image
imageFilename = "./data/images/ben.jpg"
im = cv2.imread(imageFilename)
# landmarks will be stored in results/family_i.txt
landmarksBasename = "./data/images/ben"

In [14]:
# Detect faces in the image
faceRects = faceDetector(im, 0)
print("Number of faces dectected: ", len(faceRects))

# List to store landmarks of all detected faces
landmarksAll = []

Number of faces dectected:  1


In [15]:
# Loop over all detected face rectangles
for i in range(0, len(faceRects)) :
    newRect = dlib.rectangle(int(faceRects[i].left()),
                            int(faceRects[i].top()),
                            int(faceRects[i].right()),
                            int(faceRects[i].bottom()))
    # For every face rectangle, run landmarkDectector
    landmarks = landmarkDetector(im, newRect)
    # Print number of landmarks
    if i==0:
        print("Number of landmarks", len(landmarks.parts()))
        
    # Store landmarks for current face
    landmarksAll.append(landmarks)
    
    # Draw landmarks on face
    renderFace(im, landmarks)
    
    landmarksFileName = landmarksBasename + "_" + str(i) + ".txt"
    print("Saving landmarks to", landmarksFileName)
    # Write landmarks to disk
    writeLandmarksToFile(landmarks, landmarksFileName)

Number of landmarks 68
Saving landmarks to ./data/images/ben_0.txt


In [ ]:
outputFileName = "results/t2Landmarks.jpg"
print("Saving output image to", outputFileName)
cv2.imwrite(outputFileName, im)

cv2.imshow("Facial Landmark dectector", im)
cv2.waitKey(0)
cv2.destroyAllWindows()